In [1]:
import carla
import numpy as np
import cv2
import time
import sys
import math
sys.path.append('F:\CARLA\Windows\CARLA_0.9.15\PythonAPI\carla') # tweak to where you put carla
from agents.navigation.global_route_planner import GlobalRoutePlanner
from Environment import EnvironmentClass, Learn
from RLAlgorithm import ActorCriticAgent, DDPGAgent
from datetime import datetime
import torch
from torch.nn import functional as F
import matplotlib.pyplot as plt
from torch.utils.tensorboard import SummaryWriter
import time
import os

c:\Users\mihge\anaconda3\envs\carla-sim\lib\site-packages\requests\__init__.py:114: RequestsDependencyWarning: urllib3 (1.26.14) or chardet (None)/charset_normalizer (3.1.0) doesn't match a supported version!
  RequestsDependencyWarning,


In [2]:
client = carla.Client('localhost', 2000)

world = client.get_world()

blueprints = [bp for bp in world.get_blueprint_library().filter('*')]

spawn_points = world.get_map().get_spawn_points()
print(spawn_points)

spectator = world.get_spectator()

[<carla.libcarla.Transform object at 0x000001D6E2F74490>, <carla.libcarla.Transform object at 0x000001D6E2F74500>, <carla.libcarla.Transform object at 0x000001D6E2F742D0>, <carla.libcarla.Transform object at 0x000001D6E2F74650>, <carla.libcarla.Transform object at 0x000001D6E2F746C0>, <carla.libcarla.Transform object at 0x000001D6E2F74730>, <carla.libcarla.Transform object at 0x000001D6E2F747A0>, <carla.libcarla.Transform object at 0x000001D6E2F74810>, <carla.libcarla.Transform object at 0x000001D6E2F74880>, <carla.libcarla.Transform object at 0x000001D6E2F748F0>, <carla.libcarla.Transform object at 0x000001D6E2F74960>, <carla.libcarla.Transform object at 0x000001D6E2F749D0>, <carla.libcarla.Transform object at 0x000001D6E2F74A40>, <carla.libcarla.Transform object at 0x000001D6E2F74AB0>, <carla.libcarla.Transform object at 0x000001D6E2F74B20>, <carla.libcarla.Transform object at 0x000001D6E2F74B90>, <carla.libcarla.Transform object at 0x000001D6E2F74C00>, <carla.libcarla.Transform obje

In [3]:
def destroy():#Destroying the existing things
    for actor in world.get_actors().filter('*vehicle*'):
        actor.destroy()
    for sensor in world.get_actors().filter('*sensor*'):
        sensor.destroy()
destroy()

In [4]:
def DrawPointsFor30Sec(world, spawn_points):
    drawn_points = []
    for index, waypoint in enumerate(spawn_points):
        # Draw a string with an ID at the location of each spawn point
        point_id = f'ID: {index}'
        point = world.debug.draw_string(
            waypoint.location,
            point_id,
            draw_shadow=False,
            color=carla.Color(r=255, g=255, b=255),
            life_time=30,  # Set to 0 to make it persist indefinitely
            persistent_lines=True
        )
        drawn_points.append(point)
    world.tick()
#DrawPointsFor30Sec(world, spawn_points)

In [5]:
def get_distance(location1, location2):
    return math.sqrt((location1.x - location2.x)**2 + (location1.y - location2.y)**2)

In [6]:
def setup_global_route_planner(carla_map, sampling_resolution=2.0):
    grp = GlobalRoutePlanner(carla_map, sampling_resolution)
    return grp

In [7]:
def get_route(grp, start_location, end_location):
    """
    Returns a list of waypoints from start to end using GlobalRoutePlanner.
    """
    route = grp.trace_route(start_location, end_location)
    # Each element in route is a (waypoint, RoadOption), we extract only the waypoint
    return route

In [8]:
def bicycleorigin(id):
    global bicycle
    bicycle_bp = world.get_blueprint_library().filter('*crossbike*')
    bicycle_start_point = spawn_points[id]

    bicycle = world.try_spawn_actor(bicycle_bp[0], bicycle_start_point)
    if id == 57:
        bicyclepos = carla.Transform(bicycle_start_point.location + carla.Location(x=-3, y=3.5))
        bicycle.set_transform(bicyclepos)
    elif id == 43:
        bicycle_location  = bicycle_start_point.location + carla.Location(x=-3.5, y=-7)
        bicycle_rotation  = carla.Rotation(pitch=0, yaw= bicycle_start_point.rotation.yaw + 0, roll=0)
        bicyclepos = carla.Transform(bicycle_location, bicycle_rotation)
        bicycle.set_transform(bicyclepos)
    elif id == 35:
        bicycle_location  = bicycle_start_point.location + carla.Location(x=-7, y=-3.5)
        bicycle_rotation  = carla.Rotation(pitch=0, yaw= bicycle_start_point.rotation.yaw +0, roll=0)
        bicyclepos = carla.Transform(bicycle_location, bicycle_rotation)
        bicycle.set_transform(bicyclepos)
    elif id == 79:
        bicycle_location  = bicycle_start_point.location + carla.Location(x=3.5, y=5)
        bicycle_rotation  = carla.Rotation(pitch=0, yaw= bicycle_start_point.rotation.yaw +0, roll=0)
        bicyclepos = carla.Transform(bicycle_location, bicycle_rotation)
        bicycle.set_transform(bicyclepos)
    control = carla.VehicleControl()
    control.throttle = 1.0
    control.steer = 0.0
    bicycle.apply_control(control)
    return bicycle

In [9]:
def get_offset_location(base_location, yaw, offset):
    """Rotates the offset based on yaw and adds it to the base location"""
    rad = math.radians(yaw)
    x = base_location.x + offset.x * math.cos(rad) - offset.y * math.sin(rad)
    y = base_location.y + offset.x * math.sin(rad) + offset.y * math.cos(rad)
    z = base_location.z + offset.z
    return carla.Location(x=x, y=y, z=z)

In [10]:
def follow_route(vehicle, route, world, agent: DDPGAgent, env: EnvironmentClass, cyclist_spawn_ids=None, threshold=2.0, spawn_distance=30, despawn_distance=30.0):  
    global cyclist_spawned_flags, spawned_cyclists, objectssata

    waypoints = [wp[0] for wp in route]

    if cyclist_spawn_ids is None:
        cyclist_spawn_ids = []

    for waypoint in waypoints:
        target = waypoint.transform.location

        while True:
            location = vehicle.get_location()
            distance = get_distance(location, target)
            if distance < threshold:
                break

            for i, spawn_id in enumerate(cyclist_spawn_ids):
                cyclist_location = spawn_points[spawn_id].location

                if not cyclist_spawned_flags[i]:
                    dist_to_cyclist = get_distance(location, cyclist_location)
                    if dist_to_cyclist < spawn_distance:
                        cyclist = bicycleorigin(spawn_id)
                        env.bicycle = cyclist
                        if cyclist:
                            spawned_cyclists[i] = cyclist
                            cyclist_spawned_flags[i] = True
                            print(f"🚴 Cyclist {spawn_id} spawned")
                else:
                    # If already spawned, check if it's far enough to destroy
                    cyclist = spawned_cyclists[i]
                    if cyclist is not None:
                        dist_to_vehicle = get_distance(location, cyclist.get_location())
                        if dist_to_vehicle > despawn_distance:
                            cyclist.destroy()
                            spawned_cyclists[i] = None
                            cyclist_spawned_flags[i] = False
                            print(f"💥 Cyclist {spawn_id} destroyed (too far)")

            #I guess we need the env.step here
            action = agent.choose_action(objectssata)
            # print(action)
            # # Navigation logic
            # vehicle_transform = vehicle.get_transform()
            # vehicle_location = vehicle_transform.location
            # vehicle_yaw = math.radians(vehicle_transform.rotation.yaw)

            # dx = target.x - vehicle_location.x
            # dy = target.y - vehicle_location.y
            # target_yaw = math.atan2(dy, dx)

            # steer_angle = target_yaw - vehicle_yaw
            # while steer_angle < -math.pi:
            #     steer_angle += 2 * math.pi
            # while steer_angle > math.pi:
            #     steer_angle -= 2 * math.pi

            # control = carla.VehicleControl()
            # control.throttle = 0.5
            # control.steer = max(-1.0, min(1.0, steer_angle))

            # vehicle.apply_control(control)
            #  #for view pusposes_only
            # transform = vehicle.get_transform()
            # location = transform.location
            # rotation = transform.rotation

            # # Adjust camera position (behind and above vehicle)
            # offset = carla.Location(x=-6.0, y=0.0, z=3.0)
            # camera_location = get_offset_location(location, rotation.yaw, offset)

            # world.tick()
            # spectator.set_transform(carla.Transform(camera_location, carla.Rotation(pitch=-15, yaw=rotation.yaw)))
            # loc = vehicle.get_location()
            # world.debug.draw_string(loc + carla.Location(z=2.5), "Ego Vehicle", draw_shadow=False,
            #                 color=carla.Color(255, 255, 0), persistent_lines=False)
            # world.debug.draw_string(loc + carla.Location(z=2.4), f"Control: {control.throttle}", draw_shadow=False,
            #                 color=carla.Color(255, 255, 0) , persistent_lines=False)


            returnvalues = env.step(action, 0)
            objectssata, reward, done, terminated, next_step = returnvalues

In [11]:
destroy()

In [12]:
SEED = 42

torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [13]:
LearningRateA = 0.00025
LearningRateB = 0.0025

#Initialize the environment
carenv = EnvironmentClass("Testing", SEED=SEED)
INPUTDIM = len(carenv.objectreturn)
#Initialize the agent
agent = DDPGAgent(alpha=LearningRateA, beta = LearningRateB, input_dims=[INPUTDIM], n_actions=2, seed=SEED, tau=0.001)
agent.load_models()
#Cyclist spawn points
cyclist_spawn_ids = [57, 43, 35, 79] 
cyclist_spawned_flags = [False] * len(cyclist_spawn_ids)
spawned_cyclists = [None] * len(cyclist_spawn_ids)

objectssata, reward, done, terminated, next_step, point_a, point_b, vehicle = carenv.test()
carla_map = world.get_map()
grp = setup_global_route_planner(carla_map)
print("Generating routes using GlobalRoutePlanner...")
route_ab = get_route(grp, point_a, point_b)
route_ba = get_route(grp, point_b, point_a)

full_route = route_ab + route_ba[1:]
x = 3  # Number of full round trips
full_loop_route = full_route * x

carenv.routesetup(full_loop_route)
follow_route(vehicle, full_loop_route, world, cyclist_spawn_ids=cyclist_spawn_ids, agent=agent, env=carenv)
    
destroy()
# Cleanup
cv2.destroyAllWindows()


Env torch
tensor([0.8823])
None
Python random: 0.6394267984578837
NumPy random: 0.3745401188473625
... loading checkpoint ...
... loading checkpoint ...
... loading checkpoint ...
... loading checkpoint ...
Generating routes using GlobalRoutePlanner...
🚴 Cyclist 57 spawned


f:\CARLA\Windows\CARLA_0.9.15\PythonAPI\ARI_CARLA\BaseSetup\RLAlgorithm\DDPG.py:82: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  state = T.tensor(observation, dtype=T.float).to(self.actor.device)


Throttle
0.9300941
Brake
0.0
Throttle
0.91333383
Brake
0.0
🚴 Cyclist 43 spawned
Throttle
0.9222391
Brake
0.0
Throttle
0.93580985
Brake
0.0
Throttle
0.0
Brake
0.9823387
Throttle
0.0
Brake
0.99886984
Throttle
0.9971928
Brake
0.0
Throttle
0.0
Brake
0.9785286
Throttle
0.0
Brake
0.9897267
Throttle
0.0
Brake
0.98334473
Throttle
0.0
Brake
1.0007508
Throttle
0.0
Brake
1.0054669
Throttle
0.0
Brake
1.0080723
Throttle
0.0
Brake
1.0180223
Throttle
0.0
Brake
1.0082119
Throttle
0.0
Brake
1.013378
Throttle
0.0
Brake
1.0065042
Throttle
0.0
Brake
0.9754824
Throttle
0.9549666
Brake
0.0
💥 Cyclist 57 destroyed (too far)
Throttle
1.1060332
Brake
0.0
Throttle
1.1273816
Brake
0.0
Throttle
1.1470902
Brake
0.0
Throttle
1.1252254
Brake
0.0
Throttle
0.0
Brake
1.113908
🚴 Cyclist 35 spawned
💥 Cyclist 35 destroyed (too far)
🚴 Cyclist 35 spawned
💥 Cyclist 35 destroyed (too far)
🚴 Cyclist 35 spawned
💥 Cyclist 35 destroyed (too far)
🚴 Cyclist 35 spawned
💥 Cyclist 43 destroyed (too far)
Throttle
1.085114
Brake
0.0
Thro

IndexError: list index out of range